In this notebook we plot 2-point correlation functions and light-cone OTOCs. We use the example tree-unitary gates generated by the algorithm in tu_examples, then a maximum velocity tree-unitary gate, then the kicked Ising model. We start by doing this for the specific path 2->3. At the end, we generalise this to consider arbitrary light cone paths.

In [ ]:
using DelimitedFiles
using TensorOperations
using IterTools
using LinearAlgebra
using Plots
using LaTeXStrings

In [ ]:
matrix=readdlm("tu_examples/treeunitary.txt")

tensor=reshape(matrix,(2,2,2,2,2,2));

In [ ]:
#check unitarity

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[a,b,c,l,m,n]*conj(tensor[d,e,f,l,m,n])
end
println(real.(round.(reshape(res,(8,8)),digits=5)))

#check TU condition 1
res=zeros(ComplexF64,2,2,2,2)

@tensor begin
    res[a,b,c,d]:=tensor[i,a,j,l,b,k]*conj(tensor[i,c,j,l,d,k])
end
println(round.(reshape(res,(4,4)),digits=5))

#check TU condition 2
res=zeros(ComplexF64,2,2,2,2)

@tensor begin
    res[a,b,c,d]:=tensor[a,i,j,b,l,k]*conj(tensor[c,i,j,d,l,k])
end
println(round.(reshape(res,(4,4)),digits=5))

#check TU condition 3
res=zeros(ComplexF64,2,2,2,2)

@tensor begin
    res[a,b,c,d]:=tensor[i,j,a,l,k,b]*conj(tensor[i,j,c,l,k,d])
end
println(round.(reshape(res,(4,4)),digits=5))

In [ ]:
#check max velocity condition 1

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[i,a,j,k,b,c]*conj(tensor[i,d,j,k,e,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 2

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[i,a,j,b,c,k]*conj(tensor[i,d,j,e,f,k])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 3

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[a,i,j,b,c,k]*conj(tensor[d,i,j,e,f,k])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 4

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[a,i,j,b,k,c]*conj(tensor[d,i,j,e,k,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 5

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[i,j,a,b,k,c]*conj(tensor[i,j,d,e,k,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 6

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor[i,j,a,k,b,c]*conj(tensor[i,j,d,k,e,f])
end
println(round.(reshape(res,(8,8)),digits=5))

This is an example of a tree-unitary gate which does not satisfy any maximum velocity conditions.

# Correlation Functions

In [ ]:
function construct_M(U)
    M=zeros(ComplexF64,2,2,2,2)
    @tensor begin
        M[a,b,c,d]=0.25*U[i,j,a,k,b,l]*conj(U)[i,j,c,k,d,l]
    end
    return M
end

function contract_M_op(M,op)
    res=zeros(ComplexF64,2,2)
    @tensor begin
        res[a,c]=M[a,b,c,d]*op[b,d]
    end
    return res
end

function contract_op_op(op1,op2)
    res=0
    @tensor begin
        res=op1[a,b]*op2[b,a]
    end
    return res
end

function cf_evol(U,op1,op2,nt)
    tvals=1:nt
    cfvals=zeros(ComplexF64,nt)

    M=construct_M(U)

    opt=op1

    for t in tvals
        cfvals[t]=contract_op_op(op2,opt)
        opt=contract_M_op(M,opt)
    end

    return tvals,cfvals
end

In [ ]:
X=[0 1; 1 0]
Y=[0 -im; im 0]
Z=[1 0; 0 -1]
Id=[1 0; 0 1];

In [ ]:
op_a=(1/sqrt(2))*(X + Z)
op_b=Z
tmax=13
tvals,cfvals=cf_evol(tensor,op_a,op_b,tmax);

xticks=(2:2:14,[L"%$i" for i in 2:2:14])
yticks=(-0.5:0.5:1.0,[L"%$i" for i in -0.5:0.5:1.0])
plot(tvals,real.(cfvals),xlabel=L"t",ylabel=L"c_{\alpha \beta}(x,t)",xticks=xticks,yticks=yticks,yrange=[-0.9,1.6],linewidth=2,label=false)

# OTOCs

Here we look at the OTOC in the 2->3 direction.

In [ ]:
function construct_T(U)
    #construct OTOC channel on the lightcone for 2->3 direction
    #U should have 3 inputs and 3 outputs
    # returns a tensor with 4 inputs and 4 outputs, each leg having dimension q

    T1=zeros(ComplexF64,fill(2,8)...)
    T2=zeros(ComplexF64,fill(2,10)...)
    T3=zeros(ComplexF64,fill(2,12)...)
    T4=zeros(ComplexF64,fill(2,8)...)
    T=zeros(ComplexF64,fill(2,8)...)
    @tensor begin 
        T1[a,b,c,f,g,h,i,j]=U[a,b,c,d,e,f]*conj(U)[h,i,j,d,e,g]
    end
    @tensor begin
        T2[a,b,c,f,g,i,k,l,m,n]=T1[a,b,c,f,g,h,i,j]*U[h,k,j,l,m,n]
    end
    @tensor begin
        T3[a,b,c,f,g,i,k,n,o,p,q,r]=T2[a,b,c,f,g,i,k,l,m,n]*conj(U)[p,q,r,l,m,o]
    end
    @tensor begin
        T4[b,f,g,i,k,n,o,q]=T3[a,b,c,f,g,i,k,n,o,a,q,c]
    end
    @tensor begin
        T[a,b,c,d,e,f,g,h]=T4[a,e,f,b,c,g,h,d]
    end
    return T./4

end


function construct_L(op_L)
    #construct left boundary for a given operator
    L=zeros(ComplexF64,fill(2,4)...)
    @tensor begin
        L[a,b,c,d]=op_L[b,c]*op_L[d,a]
    end
    return L/sqrt(2)
end

function construct_R(op_R)
    #construct left boundary for a given operator
    R=zeros(ComplexF64,fill(2,4)...)
    @tensor begin
        R[e,f,g,h]=op_R[e,f]*op_R[g,h]
    end
    return R/sqrt(2)
end

function mult_vec_vec(vec1,vec2)
    #contract two 'vectors' with 4 indices together
    res=0
    @tensor begin
        res=vec1[a,b,c,d]*vec2[a,b,c,d]
    end
    return res

end


function mult_tens_vec(tens,vec)
    #contract the channel tens with the vector vec
    res=zeros(ComplexF64,fill(2,4)...)
    @tensor begin
        res[a,b,c,d]=tens[a,b,c,d,e,f,g,h]*vec[e,f,g,h]
    end
    return res
end

function otoc_evol(U,op_L,op_R,nt)
    #evaluates the light cone otoc for series of time steps
    tvals=1:nt
    otocvals=zeros(ComplexF64,nt)

    T=construct_T(U)
    L=construct_L(op_L)
    R=construct_R(op_R)

    vect=R

    for t in tvals
        otocvals[t]=mult_vec_vec(L,vect)
        vect=mult_tens_vec(T,vect)
    end

    return tvals,otocvals
end


In [ ]:
op_a=(1/sqrt(2))*(X + Z)
op_b=Z

tvals,otocvals=otoc_evol(tensor,op_a,op_b,100);

xticks=(0:25:100,[L"%$i" for i in 0:25:100])
yticks=(-0.25:0.25:1.0,[L"%$i" for i in -0.25:0.25:1.0])

plt=plot(tvals,real.(otocvals),xlabel=L"t",ylabel=L"C_{\alpha \beta}(x,t)",xticks=xticks,yticks=yticks,yrange=[-0.35,1.1],linewidth=2,label=false)


# Maximum Velocity Tree-Unitary

We now consider the OTOC for a maximum velocity tree-unitary gate in the 2->3 direction.

In [ ]:
matrix=readdlm("tu_examples/treeunitary_mv.txt")
tensor_mv=permutedims(reshape(matrix,(2,2,2,2,2,2)),(6,5,4,3,2,1));

In [ ]:
#check max velocity condition 1

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor_mv[i,a,j,k,b,c]*conj(tensor_mv[i,d,j,k,e,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 2

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor_mv[i,a,j,b,c,k]*conj(tensor_mv[i,d,j,e,f,k])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 3

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor_mv[a,i,j,b,c,k]*conj(tensor_mv[d,i,j,e,f,k])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 4

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor_mv[a,i,j,b,k,c]*conj(tensor_mv[d,i,j,e,k,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 5

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor_mv[i,j,a,b,k,c]*conj(tensor_mv[i,j,d,e,k,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 6

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=tensor_mv[i,j,a,k,b,c]*conj(tensor_mv[i,j,d,k,e,f])
end
println(round.(reshape(res,(8,8)),digits=5))

In [ ]:
op_a=(1/sqrt(2))*(X + Z)
op_b=Z

tvals,otocvals=otoc_evol(tensor_mv,op_a,op_b,20);

xticks=(0:5:20,[L"%$i" for i in 0:5:20])
yticks=(-0.5:0.25:0.5,[L"%$i" for i in -0.5:0.25:0.5])
plt=plot(tvals,real.(otocvals),xlabel=L"t",ylabel=L"C_{\alpha \beta}(x,t)",xticks=xticks,yticks=yticks,yrange=[-0.6,0.3],linewidth=2,label=false)


# Kicked Ising Model

Finally we consider the OTOC for the KIM.

In [ ]:
h=0.1
hmat=[exp(-im*h)  im; im  exp(im*h)]
kmat=1/sqrt(2) * [1 -im; -im 1]

kim = zeros(ComplexF64,2,2,2,2,2,2)

for (i,j,k,l,m,n) in IterTools.product(fill(1:2,6)...)
    kim[i,j,k,l,m,n]=hmat[i,j]*hmat[j,k]*kmat[i,l]*kmat[j,m]*kmat[k,n]*hmat[l,m]*hmat[m,n]
end

In [ ]:
#check unitarity

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[a,b,c,l,m,n]*conj(kim[d,e,f,l,m,n])
end
println(real.(round.(reshape(res,(8,8)),digits=5)))

#check TU condition 1
res=zeros(ComplexF64,2,2,2,2)

@tensor begin
    res[a,b,c,d]:=kim[i,a,j,l,b,k]*conj(kim[i,c,j,l,d,k])
end
println(round.(reshape(res,(4,4)),digits=5))

#check TU condition 2
res=zeros(ComplexF64,2,2,2,2)

@tensor begin
    res[a,b,c,d]:=kim[a,i,j,b,l,k]*conj(kim[c,i,j,d,l,k])
end
println(round.(reshape(res,(4,4)),digits=5))

#check TU condition 3
res=zeros(ComplexF64,2,2,2,2)

@tensor begin
    res[a,b,c,d]:=kim[i,j,a,l,k,b]*conj(kim[i,j,c,l,k,d])
end
println(round.(reshape(res,(4,4)),digits=5))

In [ ]:
#check max velocity condition 1

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[i,a,j,k,b,c]*conj(kim[i,d,j,k,e,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 2

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[i,a,j,b,c,k]*conj(kim[i,d,j,e,f,k])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 3

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[a,i,j,b,c,k]*conj(kim[d,i,j,e,f,k])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 4

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[a,i,j,b,k,c]*conj(kim[d,i,j,e,k,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 5

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[i,j,a,b,k,c]*conj(kim[i,j,d,e,k,f])
end
println(round.(reshape(res,(8,8)),digits=5))

#check max velocity condition 6

res=zeros(ComplexF64,2,2,2,2,2,2)

@tensor begin
    res[a,b,c,d,e,f]:=kim[i,j,a,k,b,c]*conj(kim[i,j,d,k,e,f])
end
println(round.(reshape(res,(8,8)),digits=5))

In [ ]:
op_a=(1/sqrt(2))*(X + Z)
op_b=Y
op_c=(1/sqrt(2))*(Y + Z)
# op_a=X
# op_b=X
tvals,otocvals=otoc_evol(kim,op_a,op_b,40);
tvals,otocvals2=otoc_evol(kim,op_a,op_c,40);

plt=plot(tvals,real.(otocvals),xlabel=L"t",ylabel=L"C_{\alpha \beta}(x,t)",xticks=xticks,yticks=yticks,yrange=[-1.05,0.05],linewidth=2,label=L"\textrm{KIM},~\sigma_{\beta}=Y",legend=:bottomright, legendfontsize=6)
plot!(tvals,real.(otocvals2),label=L"\textrm{KIM},~\sigma_{\beta}=\frac{1}{\sqrt{2}}(Y+Z)",linewidth=2)


# Arbitrary Light Cone Paths

Here we implement the general channel M_{ij}.

In [ ]:
function construct_M(U, n, m)

    if !(n in 1:3) || !(m in 1:3)
        error("n and m must be in the range 1:3")
    end

    # original index groups:
    first_block  = [1,2,3]
    second_block = [4,5,6]

    # determine free and contracted indices for first block
    free_first = n          # free index is in position n
    contr_first = filter(x -> x != n, first_block)
    # println(contr_first)

    # for second block, free index is at position m+3
    free_second = m + 3
    contr_second = filter(x -> x != free_second, second_block)

    # build the permutation vector
    # new ordering: [free_first, contr_first..., free_second, contr_second...]
    perm = vcat([free_first], contr_first, [free_second], contr_second)

    # permute U
    U_perm = permutedims(U, perm)


    dimA = size(U, free_first)
    dimB = size(U, free_second)

    # initialize M with dimensions (dimA, dimB, dimA, dimB)
    M = zeros(ComplexF64, dimA, dimB, dimA, dimB)

    # now contract over the contracted indices (positions 2,3 and 5,6).
    # free indices in U_perm (positions 1 and 4) are renamed to a and b, and similarly for conj(U_perm) we use c and d.
    @tensor M[a, b, c, d] := 0.25 * U_perm[a, i, j, b, k, l] * conj(U_perm)[c, i, j, d, k, l]

    return M
end

We can check that this agrees with the previous channel:

In [ ]:
construct_M(tensor,3,2)-construct_M(tensor)

To evaluate light cone correlation functions, we need to define the path as a sequence of legs on the unitaries e.g. ```upath=[[2,1],[2,1],[2,3]]```

In [ ]:
function cf_evol_path(U,op1,op2,upath)
    # tvals=1:length(path)
    cfvals=zeros(ComplexF64,length(upath))

    opt=op1

    for i in 1:length(upath)
        M=construct_M(U,upath[i]...)
        opt=contract_M_op(M,opt)
        cfvals[i]=contract_op_op(op2,opt)
        
    end

    return cfvals
end

Finally, for a giving starting node and a finishing node, we produce a function to find the relevant path and evaluate the light cone correlation function.

We want to find this upath for a given pair of nodes, which will define a path through the graph.

The first step in doing this is to find the sequence of nodes we pass through:

In [ ]:
function build_graph(triples)
    graph = Dict{Int, Set{Int}}()
    # Combine the triple sets
    for triple in triples
        a, b, c = triple
        # For each triple, add edges between a and b and between b and c (undirected)
        for (u, v) in ((a, b), (b, c))
            if !haskey(graph, u)
                graph[u] = Set{Int}()
            end
            if !haskey(graph, v)
                graph[v] = Set{Int}()
            end
            push!(graph[u], v)
            push!(graph[v], u)
        end
    end
    return graph
end

function find_path(graph::Dict{Int, Set{Int}}, start::Int, goal::Int)
    # Dictionary to record each node's parent in the BFS tree.
    parent = Dict{Int, Int}()
    visited = Set{Int}()
    queue = [start]
    push!(visited, start)
    
    while !isempty(queue)
        node = popfirst!(queue)
        if node == goal
            # Goal found, reconstruct the path by backtracking.
            path = [goal]
            while path[1] != start
                pushfirst!(path, parent[path[1]])
            end
            return path
        end
        for neighbor in graph[node]
            if neighbor ∉ visited
                push!(visited, neighbor)
                parent[neighbor] = node
                push!(queue, neighbor)
            end
        end
    end
    return nothing  # If no path is found.
end


function compress_path(path::Vector{Int}, triples)
    path = copy(path)  # Work on a copy so as not to modify the original
    changed = true
    while changed
        changed = false
        # Check every consecutive triple in the path.
        for i in 1:(length(path)-2)
            triple_candidate = (path[i], path[i+1], path[i+2])
            for t in triples
                # Check if candidate matches t or its reverse.
                if triple_candidate == t || triple_candidate == (t[3], t[2], t[1])
                    deleteat!(path, i+1)  # Remove the middle node
                    changed = true
                    break  # Break out of the inner loop.
                end
            end
            if changed
                break  # Restart the while loop since the path changed.
            end
        end
    end
    return path
end


function compressed_path_from_nodes(in,out,triples)
    graph = build_graph(triples)
    full_path=find_path(graph, in, out)
    return compress_path(full_path,triples)
end

In [ ]:
include("src/TreeStructure.jl")

In [ ]:
triplets=graph_nodes_zsites(3,4)
compressed_path_from_nodes(1,41,vcat(triplets[1],triplets[2]))

Next we need to figure out which paths through the *unitaries* this corresponds to, to turn this into a path through the unitaries of the form [1,2],[1,3],[2,1]...

In [ ]:
# Function to find the positions of two nodes in a triple
function find_triple_positions(a, b, triples)
    for triple in triples
        # Check if both nodes are present in the triple
        if a in triple && b in triple
            # Find the positions (indices) of a and b within the triple
            posA = findfirst(x -> x == a, triple)
            posB = findfirst(x -> x == b, triple)
            return (posA, posB)
        end
    end
    return nothing  # Return nothing if no matching triple is found
end

function position_pairs(path,triples)
    return [find_triple_positions(path[i], path[i+1], triples) for i in 1:(length(path)-1)]
end

function cf_path_val(U,op1,op2,path,triples)
    # tvals=1:length(path)
    # cfvals=zeros(ComplexF64,length(path))
    # here path is just list of nodes
    opt=op1
    upath=position_pairs(path,triples)

    for i in 1:length(upath)
        M=construct_M(U,upath[i]...)
        opt=contract_M_op(M,opt)
        # cfvals[i]=contract_op_op(op2,opt)
        
    end

    return contract_op_op(op2,opt)
end

In [ ]:
op_a=(1/sqrt(2))*(X + Z)
op_b=Z

triplets=graph_nodes_zsites(3,4)
triples=vcat(triplets[1],triplets[2])
start=1
stop=41
path=compressed_path_from_nodes(start,stop,triples)
println(position_pairs(path,triples))
cf_path_val(tensor, op_a, op_b, path, triples)
